In [ ]:
#!pip install -- upgrade openrouteservice

In [27]:
import openrouteservice
import folium
import pandas as pd
from geopy.distance import geodesic
import json
import requests

def integrated_ev_route_planner(origin, destination, max_range, battery_size, openrouteservice_api_key, openchargemap_api_key):
    client = openrouteservice.Client(key=openrouteservice_api_key)

    def geocode(location):
        try:
            result = client.pelias_search(text=location)
            if 'features' in result and result['features']:
                coordinates = result['features'][0]['geometry']['coordinates']
                return coordinates  # [lon, lat]
            else:
                raise Exception(f"No coordinates found for {location}")
        except Exception as e:
            raise Exception(f"Error geocoding {location}: {str(e)}")

    def calculate_battery_consumed(distance, max_range, battery_size):
        energy_consumption_rate = battery_size / max_range
        consumed_energy = energy_consumption_rate * distance
        percent_consumed_energy = consumed_energy / battery_size * 100
        return percent_consumed_energy

    def find_charging_stations(lat, lon, radius):
        url = f"https://api.openchargemap.io/v3/poi/?output=json&latitude={lat}&longitude={lon}&distance={radius}&distanceunit=KM&maxresults=10&key={openchargemap_api_key}"
        response = requests.get(url)
        if response.status_code == 200:
            stations = response.json()
            return stations
        else:
            print(f"Error fetching charging stations: {response.status_code}")
            return []

    def find_stops(route_coordinates, max_range, buffer_range):
        stops = []
        distance_traveled = 0
        current_location = route_coordinates[0]

        for i in range(1, len(route_coordinates)):
            segment_distance = geodesic(current_location[::-1], route_coordinates[i][::-1]).km
            distance_traveled += segment_distance

            if distance_traveled > max_range - buffer_range:
                charging_stations = find_charging_stations(current_location[1], current_location[0], 20)  # 20km radius

                if charging_stations:
                    closest_station = min(charging_stations, key=lambda s: 
                        geodesic((s['AddressInfo']['Latitude'], s['AddressInfo']['Longitude']), current_location[::-1]).km)
                    stops.append({
                        "name": closest_station['AddressInfo']['Title'],
                        "lon": closest_station['AddressInfo']['Longitude'],
                        "lat": closest_station['AddressInfo']['Latitude']
                    })
                    current_location = [closest_station['AddressInfo']['Longitude'], closest_station['AddressInfo']['Latitude']]
                    distance_traveled = 0
                else:
                    print(f"Warning: No charging stations found near {current_location}")

            current_location = route_coordinates[i]

        return stops

    def plot_route_on_map(origin_input, origin_coords, destination_input, destination_coords, ev_stations, route):
        route_coordinates = route['geometry']['coordinates']
        
        map_center = [(origin_coords[1] + destination_coords[1]) / 2, (origin_coords[0] + destination_coords[0]) / 2]
        map_obj = folium.Map(location=map_center[::-1], zoom_start=10, width=380, height=700)

        lats = [coord[1] for coord in route_coordinates] + [origin_coords[1], destination_coords[1]]
        lons = [coord[0] for coord in route_coordinates] + [origin_coords[0], destination_coords[0]]
        min_lat, max_lat = min(lats), max(lats)
        min_lon, max_lon = min(lons), max(lons)

        map_obj.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
        
        folium.Marker(location=origin_coords[::-1], popup=f'Origin: {origin_input}', icon=folium.Icon(color='green',icon='home', prefix='fa')).add_to(map_obj)
        folium.Marker(location=destination_coords[::-1], popup=f'Destination: {destination_input}', icon=folium.Icon(color='blue',icon='location-pin', prefix='fa')).add_to(map_obj)
        
        for stop in ev_stations:
            folium.Marker(
                location = [stop['lat'], stop['lon']],
                popup = stop['name'],
                icon = folium.Icon(color='red',icon='charging-station', prefix='fa')
            ).add_to(map_obj)
        
        folium.PolyLine(locations=[coord[::-1] for coord in route_coordinates], color='blue', weight=3, opacity=0.7).add_to(map_obj)

        properties = route.get('properties', {})
        segments = properties.get('segments', [])
        
        table_data = []
        locations = [f'{origin_input} (Origin)'] + [station['name'] for station in ev_stations] + [f'{destination_input} (Destination)']
        
        cumulative_distance = 0
        cumulative_duration = 0
        
        for i, location in enumerate(locations):
            if i > 0 and i-1 < len(segments):
                segment = segments[i-1]
                cumulative_distance += segment.get('distance', 0) / 1000  # Convert to km
                cumulative_duration += segment.get('duration', 0) / 60  # Convert to minutes
            
            table_data.append({
                'Point': location,
                'Cumulative Distance (km)': round(cumulative_distance, 2),
                'Cumulative Duration (mins)': round(cumulative_duration, 1)
            })
        
        df = pd.DataFrame(table_data)
        
        return map_obj, df

    # Main execution
    try:
        origin_coords = geocode(origin)
        destination_coords = geocode(destination)

        route_params = {
            'coordinates': [origin_coords, destination_coords],
            'profile': 'driving-car',
            'format': 'geojson'
        }
        route_result = client.directions(**route_params)

        if not isinstance(route_result, dict) or 'features' not in route_result or not route_result['features']:
            raise Exception(f"Unexpected route result format: {type(route_result)}")
        
        route = route_result['features'][0]
        properties = route.get('properties', {})
        segments = properties.get('segments', [])
        if not segments:
            raise Exception("No route segments found in the API response")
        
        total_distance = sum(segment.get('distance', 0) for segment in segments) / 1000  # Convert to km
        total_battery_consumed = calculate_battery_consumed(total_distance, max_range, battery_size)

        print(f"Total route distance: {total_distance:.2f} km")
        print(f"Vehicle range: {max_range} km")
        print(f"Battery size: {battery_size} kWh")
        print(f"Estimated total battery consumption: {total_battery_consumed:.2f}%")

        ev_stations = find_stops(route['geometry']['coordinates'], max_range, 50)  # 50 km buffer
        print(f"Found {len(ev_stations)} charging stops")

        if ev_stations:
            waypoints = [origin_coords] + [[stop['lon'], stop['lat']] for stop in ev_stations] + [destination_coords]
            route_with_stops_params = {
                'coordinates': waypoints,
                'profile': 'driving-car',
                'format': 'geojson'
            }
            route_with_stops_result = client.directions(**route_with_stops_params)

            if not isinstance(route_with_stops_result, dict) or 'features' not in route_with_stops_result or not route_with_stops_result['features']:
                raise Exception(f"Unexpected route with stops result format: {type(route_with_stops_result)}")
            route_with_stops = route_with_stops_result['features'][0]
        else:
            route_with_stops = route

        route_map, route_data = plot_route_on_map(origin, origin_coords, destination, destination_coords, ev_stations, route_with_stops)

        print(f"Total distance: {total_distance:.2f} km")
        print(f"Total battery consumed: {total_battery_consumed:.2f}%")
        print("\nRoute details:")
        print(route_data)

        return route_map, route_data, total_distance, total_battery_consumed

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, 0, 0

In [28]:
# Example usage:
origin = "Melbourne CBD, VIC, Australia"
destination = "Geelong, VIC, Australia"
max_range = 100  # km 
battery_size = 120  # kWh
openrouteservice_api_key = "YOUR_API_KEY"
openchargemap_api_key = "YOUR_API_KEY"

route_map, route_data, total_distance, total_battery_consumed = integrated_ev_route_planner(
    origin, destination, max_range, battery_size, openrouteservice_api_key, openchargemap_api_key
)

route_map


Total route distance: 74.47 km
Vehicle range: 100 km
Battery size: 120 kWh
Estimated total battery consumption: 74.47%
Found 1 charging stops
Total distance: 74.47 km
Total battery consumed: 74.47%

Route details:
                                    Point  Cumulative Distance (km)  \
0  Melbourne CBD, VIC, Australia (Origin)                      0.00   
1                    Geelong Supercharger                     63.48   
2   Geelong, VIC, Australia (Destination)                     74.71   

   Cumulative Duration (mins)  
0                         0.0  
1                        52.4  
2                        70.8  
